In [ ]:
-- Create a new table to hold the combined data
CREATE TABLE combined_weather_data (
    weather_date Text,
    weather_tavg Text,
    weather_tmin FLOAT,
    weather_tmax FLOAT,
    weather_prcp Text,
    weather_snow Text,
    weather_wdir Text,
    weather_wspd Text,
    weather_wpgt Text,
    weather_pres Text,
    weather_tsun Text,
    station_id Text,
    weather_station_name TEXT
);

INSERT INTO combined_weather_data
SELECT * FROM "weather_1995-1996";

-- Insert data from the second dataset (including 1996-01-01)
INSERT INTO combined_weather_data
SELECT * FROM "weather_1996-2005";

INSERT INTO combined_weather_data
SELECT * FROM "weather_2005-2013";

INSERT INTO combined_weather_data
SELECT * FROM "weather_2014-2015";

---count of rows in weather_1995-1996
SELECT count (*)
FROM "weather_1995-1996";
